In [32]:
import nltk
import pdb
from nltk.corpus import stopwords
import numpy as np
import pandas as pd
import string
from nltk.tokenize import TweetTokenizer
from os import getcwd
from utils import process_tweet, lookup

In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
!pip install opendatasets -q

In [4]:
import opendatasets as od
#{"username":"khwrali","key":"a2b23dfbac2443ab4db34e48318ce4ff"}
od.download("https://www.kaggle.com/datasets/karthickveerakumar/spam-filter")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: khwrali
Your Kaggle Key: ··········


100%|██████████| 2.86M/2.86M [00:00<00:00, 41.2MB/s]

In [14]:
df=pd.read_csv('/content/spam-filter/emails.csv')
print(df.head(2))
#Checking Missing values
print(df.isnull().sum())
#Checking number of spam and non_spam mails
print(df['spam'].value_counts())

                                                text  spam
0  Subject: naturally irresistible your corporate...     1
1  Subject: the stock trading gunslinger  fanny i...     1
text    0
spam    0
dtype: int64
0    4360
1    1368
Name: spam, dtype: int64


In [15]:
print(df[['text', 'spam']].sample(10))

                                                   text  spam
1639  Subject: rodrigo lamas - best wishes  i would ...     0
89    Subject: i think you might be interested  hell...     1
24    Subject: rely on us for your online prescripti...     1
1569  Subject: re : yo !  vince ,  here is a little ...     0
4717  Subject: 21 st annual energy symposium - novem...     0
2369  Subject: re : mscf speaker series  pierre - ph...     0
3043  Subject: a resume for your consideration  vinc...     0
3629  Subject: re : confirmation of meeting  i would...     0
1602  Subject: texas a & m at galveston  dr . bill m...     0
2487  Subject: eol trade size discrepancies  vince -...     0


In [16]:
# Initialize two empty lists to store spam and non_spam mails
spam = []
non_spam = []

# Iterate through the DataFrame and separate reviews based on sentiment
for _, row in df.iterrows():
    if row['spam'] == 1:
        spam.append(row['text'])
    elif row['spam'] == 0:
        non_spam.append(row['text'])

# Print or use the positive_reviews and negative_reviews lists as needed
print("Spam Mails:")
print(spam[:1])


print("\nNon_Spam Mails:")
print(non_spam[:1])

Spam Mails:
["Subject: naturally irresistible your corporate identity  lt is really hard to recollect a company : the  market is full of suqgestions and the information isoverwhelminq ; but a good  catchy logo , stylish statlonery and outstanding website  will make the task much easier .  we do not promise that havinq ordered a iogo your  company will automaticaily become a world ieader : it isguite ciear that  without good products , effective business organization and practicable aim it  will be hotat nowadays market ; but we do promise that your marketing efforts  will become much more effective . here is the list of clear  benefits : creativeness : hand - made , original logos , specially done  to reflect your distinctive company image . convenience : logo and stationery  are provided in all formats ; easy - to - use content management system letsyou  change your website content and even its structure . promptness : you  will see logo drafts within three business days . affordabili

In [17]:
print(len(spam))
print(len(non_spam))

1368
4360


In [19]:
# split the data into two pieces, one for training and one for testing (validation set)
test_spam = spam[1200:]
train_spam = spam[:1200]
test_non_spam = non_spam[4000:]
train_non_spam = non_spam[:4000]

train_x = train_spam + train_non_spam
test_x = test_spam + test_non_spam

In [20]:
train_y = np.append(np.ones(len(train_spam)), np.zeros(len(train_non_spam)))
test_y = np.append(np.ones(len(test_spam)), np.zeros(len(test_non_spam)))

In [24]:
def count_mails(result, mails, ys):
    for y, mail in zip(ys, mails):
        for word in process_tweet(mail):
            pair = (word,y)
            if pair in result:
                result[pair] += 1
            else:
                result[pair] = 1
    return result

In [25]:
freqs = count_mails({}, train_x, train_y)

In [26]:
def train_naive_bayes(freqs, train_x, train_y):
    loglikelihood = {}
    logprior = 0
    vocab = set([pair[0] for pair in freqs.keys()])
    V = len(vocab)
    N_pos = N_neg = V_pos = V_neg = 0
    for pair in freqs.keys():
        if pair[1] > 0:
            V_pos += 1
            N_pos += freqs[pair]
        else:
            V_neg += 1
            N_neg += freqs[pair]
    D = len(train_y)
    D_pos = (len(list(filter(lambda x: x > 0, train_y))))
    D_neg = (len(list(filter(lambda x: x <= 0, train_y))))
    logprior = np.log(D_pos) - np.log(D_neg)
    for word in vocab:
        freq_pos = lookup(freqs,word,1)
        freq_neg = lookup(freqs,word,0)
        p_w_pos = (freq_pos + 1) / (N_pos + V)
        p_w_neg = (freq_neg + 1) / (N_neg + V)
        loglikelihood[word] = np.log(p_w_pos/p_w_neg)
    return logprior, loglikelihood


In [27]:
logprior, loglikelihood = train_naive_bayes(freqs, train_x, train_y)
print(logprior)
print(len(loglikelihood))

-1.203972804325936
28083


In [28]:
def naive_bayes_predict(mail, logprior, loglikelihood):
    word_l = process_tweet(mail)
    p = 0
    p += logprior
    for word in word_l:
        if word in loglikelihood:
            p += loglikelihood[word]
    return p

In [29]:
def test_naive_bayes(test_x, test_y, logprior, loglikelihood):
    accuracy = 0
    y_hats = []
    for mail in test_x:
        if naive_bayes_predict(mail, logprior, loglikelihood) > 0:
            y_hat_i = 1
        else:
            y_hat_i = 0
        y_hats.append(y_hat_i)
    error = np.mean(np.absolute(y_hats-test_y))
    accuracy = 1-error
    return accuracy

In [30]:
print("Naive Bayes accuracy = %0.4f" %(test_naive_bayes(test_x, test_y, logprior, loglikelihood)))

Naive Bayes accuracy = 0.9905


In [31]:
my_mail = "Re: Machine Learning Internship - Khawar Ali applied on Indeed"
p = naive_bayes_predict(my_mail, logprior, loglikelihood)
if p>0:
  print(p,"The mail is spam")
else:
  print(p,"The review is not spam")

-2.29814749847631 The review is not spam
